In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import requests
import bs4 as bs
from os import listdir
from os.path import isfile, join
import os
import io

# Load and Save Statement and Statistics for Mt5

### Load Statement list ids from MQL

In [4]:
total_pages = 35
mt5idlist=[]
for i in range(1, total_pages):
    url = 'https://www.mql5.com/en/signals/mt5/list/page' + str(i)
    r = requests.get(url)
    htmlContent = r.content
    soup = bs.BeautifulSoup(htmlContent,'html.parser')

    ids = soup.find_all("div", class_="button button_white-and-green")
    mt5idlist.extend( [p['data-id'] for p in ids])
    time.sleep(random.randint(3, 5))

print(len(mt5idlist))

1605


In [5]:
mypath = "./Data/mt5/"
headers = {'cookie': 'lang=en; _fz_uniq=5046613820310397396; sid=CfDJ8O2AwCds3ABPuSmyNNtvSt59939oGNEye%2FAwZ1mc%2Bh8DRzt0jnReqBNR524tV5qZIsr8GVykBZl2Z4zVTBU6sxte3NKv46eSA7DQm92uYH2YWEHBdd76%2Fy7Y8k4Pzmrgkxcf9b8E0yZM3idvhMJ2yoVo9KVTdZUjCkCIMoTN6qkS; _fz_fvdt=1586144823; mt5_download_button=1613974670; _media_uuid=3691831371; auth=41yAG1RvddnKpurvZe6_JvusfPWFb0ci-HeJqQV2Rf8nOX0seYrMwizlRaa3j802MeX9_WxL6YM2NSh8YR3zTmgdMjyO23STt8e4rWIoRUxgiQs_nF_Hovy-tVjyGubYZKlNT9suLWjkUKfDSfppVQ; _fz_ssn=1658294788431610449'}



# # create DataFrame for statement
# if not os.path.isfile(mypath + "statement.csv"):    
#     url = 'https://www.mql5.com/en/signals/'+ idlist[0] + '/export/positions'
#     response = requests.get(url, headers=headers)

#     state_df = pd.read_csv(io.BytesIO(response.content), delimiter=';')
#     state_df.insert(loc=0, column='id', value=int(idlist[0]))

#     state_df.to_csv(mypath + "statement.csv")
#     time.sleep(random.randint(3, 10))

# create DataFrame for statistics
if not os.path.isfile(mypath + "statistics.csv"):    
    url = 'https://www.mql5.com/en/signals/'+ mt5idlist[0] + '?source=Site+Signals+MT5+Tile'
    r = requests.get(url)
    htmlContent = r.content
    soup = bs.BeautifulSoup(htmlContent,'html.parser')

    labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
    values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

    cols = ['id']
    for i in range(len(labels)):
        cols.append( labels[i].strip().split(':')[0] )
    statistics_df = pd.DataFrame(columns=list(dict.fromkeys(cols)))
    statistics_df.to_csv(mypath + "statistics.csv")
    time.sleep(random.randint(3, 10))

#-------------------------------------------------------------------------------
fileIds = [f.split('.')[0] for f in listdir(mypath) if isfile(join(mypath, f))]
statment_count = len(fileIds)
# statistics_df = pd.read_csv(mypath + "statistics.csv", index_col=[0])
# state_count = len(statistics_df)
# statistics_df
for id in mt5idlist:
    # pass
    # load mt5 statements
    if id not in fileIds:
        url = 'https://www.mql5.com/en/signals/' + id + '/export/positions'
        response = requests.get(url, headers=headers)  
        csv_file = open( mypath + id + '.csv', 'wb')
        csv_file.write(response.content)
        csv_file.close()  
    
        print ('Number of statement Loaded: ', len([f.split('.')[0] for f in listdir(mypath) if isfile(join(mypath, f))]), end="\r")
        time.sleep(random.randint(3, 10))

        
# fileIds = [f.split('.')[0] for f in listdir(mypath) if isfile(join(mypath, f))]
# statment_count = len(fileIds)

# state_df = pd.read_csv(mypath + "statement.csv", index_col=[0])
# for id in mt5idlist:
#     # pass
#     # load mt5 statements
#     if int(id) not in  state_df.id.values:
#         url = 'https://www.mql5.com/en/signals/' + id + '/export/positions'
#         response = requests.get(url, headers=headers)  
        
#         temp = pd.read_csv(io.BytesIO(response.content), delimiter=';')
#         temp.insert(loc=0, column='id', value=int(id))

#         state_df = pd.concat([state_df, temp])
        
#         state_df.to_csv(mypath + "statement.csv")
    
#         print ('Loaded: ', len(np.unique(state_df.id.values)), end="\r")
#         time.sleep(random.randint(3, 9))
    
    # load mt5 Statistics
    statistics_df = pd.read_csv(mypath + "statistics.csv", index_col=[0])
    if int(id) not in  statistics_df.id.values:
        url = 'https://www.mql5.com/en/signals/' + id + '?source=Site+Signals+MT5+Tile'
        response = requests.get(url, headers=headers)  
        htmlContent = response.content
        soup = bs.BeautifulSoup(htmlContent,'html.parser')

        labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
        values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

        dic = {}
        dic['id'] = int(id)
        for i in range(len(labels)):
            dic[labels[i].strip().split(':')[0]] = values[i].strip()
        temp_df = pd.DataFrame(dic, index=[0])
        statistics_df = pd.concat([statistics_df, temp_df])
        
        statistics_df.to_csv(mypath + "statistics.csv")
        
        print ('Number of statistics Loaded: ', len(statistics_df), end="\r")
        time.sleep(random.randint(3, 10))    

        


# Load and Save Statement and Statistics fo Mt4

## Load Statement list ids from MQL

In [19]:
total_pages = 91
mt4idlist=[]
for i in range(1, total_pages):
    url = 'https://www.mql5.com/en/signals/mt4/list/page' + str(i)
    r = requests.get(url)
    htmlContent = r.content
    soup = bs.BeautifulSoup(htmlContent,'html.parser')

    ids = soup.find_all("div", class_="button button_white-and-green")
    mt4idlist.extend( [p['data-id'] for p in ids])
    time.sleep(random.randint(3, 5))
    
mt4idlist = list(dict.fromkeys(mt4idlist))
print(len(mt4idlist))

4320


In [ ]:
mypath = "./Data/mt4/"
headers = {'cookie': 'lang=en; _fz_uniq=5046613820310397396; sid=CfDJ8O2AwCds3ABPuSmyNNtvSt59939oGNEye%2FAwZ1mc%2Bh8DRzt0jnReqBNR524tV5qZIsr8GVykBZl2Z4zVTBU6sxte3NKv46eSA7DQm92uYH2YWEHBdd76%2Fy7Y8k4Pzmrgkxcf9b8E0yZM3idvhMJ2yoVo9KVTdZUjCkCIMoTN6qkS; _fz_fvdt=1586144823; mt5_download_button=1613974670; _media_uuid=3691831371; auth=41yAG1RvddnKpurvZe6_JvusfPWFb0ci-HeJqQV2Rf8nOX0seYrMwizlRaa3j802MeX9_WxL6YM2NSh8YR3zTmgdMjyO23STt8e4rWIoRUxgiQs_nF_Hovy-tVjyGubYZKlNT9suLWjkUKfDSfppVQ; _fz_ssn=1658294788431610449'}

# create DataFrame for statistics
if not os.path.isfile(mypath + "statistics.csv"):    
    url = 'https://www.mql5.com/en/signals/'+ idlist[0] + '?source=Site+Signals+MT5+Tile'
    r = requests.get(url)
    htmlContent = r.content
    soup = bs.BeautifulSoup(htmlContent,'html.parser')

    labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
    values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

    cols = ['id']
    for i in range(len(labels)):
        cols.append( labels[i].strip().split(':')[0] )
    statistics_df = pd.DataFrame(columns=list(dict.fromkeys(cols)))
    statistics_df.to_csv(mypath + "statistics.csv")
    time.sleep(random.randint(3, 10))



fileIds = [f.split('.')[0] for f in listdir(mypath) if isfile(join(mypath, f))]
statment_count = len(fileIds)
state_count = len(statistics_df)

for id in mt4idlist:
    # pass
    # load mt5 statements
    if id not in fileIds:
        url = 'https://www.mql5.com/en/signals/' + id + '/export/history'
        response = requests.get(url, headers=headers)  
        csv_file = open( mypath + id + '.csv', 'wb')
        csv_file.write(response.content)
        csv_file.close()  
    
        print ('Loaded: ', statment_count, end="\r")
        statment_count += 1    
        time.sleep(random.randint(3, 10))
    
    # load mt5 Statistics
    statistics_df = pd.read_csv(mypath + "statistics.csv", index_col=[0])
    if int(id) not in  statistics_df.id.values:
        url = 'https://www.mql5.com/en/signals/' + id + '?source=Site+Signals+MT5+Tile'
        response = requests.get(url, headers=headers)  
        htmlContent = response.content
        soup = bs.BeautifulSoup(htmlContent,'html.parser')

        labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
        values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

        dic = {}
        dic['id'] = int(id)
        for i in range(len(labels)):
            dic[labels[i].strip().split(':')[0]] = values[i].strip()
        temp_df = pd.DataFrame(dic, index=[0])
        statistics_df = pd.concat([statistics_df, temp_df])
        
        statistics_df.to_csv(mypath + "statistics.csv")
        
        print ('Loaded: ', len(statistics_df), end="\r")
        time.sleep(random.randint(3, 10))    


--------------

### Test

In [93]:
url = 'https://www.mql5.com/en/signals/1431540?source=Site+Signals+MT5+Tile'
r = requests.get(url)
htmlContent = r.content
soup = bs.BeautifulSoup(htmlContent,'html.parser')

labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

In [106]:
# create DataFrame
url = 'https://www.mql5.com/en/signals/1431540?source=Site+Signals+MT5+Tile'
r = requests.get(url)
htmlContent = r.content
soup = bs.BeautifulSoup(htmlContent,'html.parser')

labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

cols = ['id']
for i in range(len(labels)):
    cols.append( labels[i].strip().split(':')[0] )
df = pd.DataFrame(columns=list(dict.fromkeys(cols)))
df

,id,Trades,Profit Trades,Loss Trades,Best trade,Worst trade,Gross Profit,Gross Loss,Maximum consecutive wins,Maximal consecutive profit,...,Average Loss,Maximum consecutive losses,Maximal consecutive loss,Monthly growth,Annual Forecast,Algo trading,Absolute,Maximal,By Balance,By Equity


In [107]:
headers = {'cookie': '_fz_uniq=5044931279740993001; _fz_fvdt=1641843692; utm_source=web.installer; utm_campaign=mql5.welcome.open; auth=41yAG1RvddnKpurvZe6_JvusfPWFb0ci-HeJqQV2Rf_kFNKZ8py9uQsT22Xv9Xy5pc2Kbg0mXQN1rfpSYKfAFeynF_fvj3j6hOOQ-AV7KhBDDRD56SqEkbRLDIQb_GFxCBkr3Ho7C4dVHqFxKiUdqA; lang=en; sid=CfDJ8O2AwCds3ABPuSmyNNtvSt6WryiUlauSGF3Omd3riSC9WE%2FHe9%2FrciGEP8ZCl2K24QrTLCd8tkU%2BrywuW2AeMGKlzo4xxba4JgJy45%2FU1lx0515Y2zy6p3z38kMPTteePyO6iisFJMfkKXkyr5tw8vk5iMxd1FWn%2BWVbLwmD%2Bsr5'}  

# count = len(df)
# df2 = None
for id in idlist[:1]:
    if int(id) in  df.id.values:
        continue
    url = 'https://www.mql5.com/en/signals/' + '843451' + '?source=Site+Signals+MT5+Tile'
    response = requests.get(url, headers=headers)  
    htmlContent = response.content
    soup = bs.BeautifulSoup(htmlContent,'html.parser')

    labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
    values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

    dic = {}
    dic['id'] = int(id)
    for i in range(len(labels)):
        dic[labels[i].strip().split(':')[0]] = values[i].strip()
    df1 = pd.DataFrame(dic, index=[0])
    df = pd.concat([df, df1])# , ignore_index=True)
   
    # count += 1    
    print ('Loaded: ', len(df), end="\r")
    time.sleep(random.randint(3, 10))



In [145]:
type(df['Profit'].values)

numpy.ndarray

In [34]:
mypath = "./Data/"
headers = {'cookie': 'lang=en; _fz_uniq=5046613820310397396; sid=CfDJ8O2AwCds3ABPuSmyNNtvSt59939oGNEye%2FAwZ1mc%2Bh8DRzt0jnReqBNR524tV5qZIsr8GVykBZl2Z4zVTBU6sxte3NKv46eSA7DQm92uYH2YWEHBdd76%2Fy7Y8k4Pzmrgkxcf9b8E0yZM3idvhMJ2yoVo9KVTdZUjCkCIMoTN6qkS; _fz_fvdt=1586144823; mt5_download_button=1613974670; _media_uuid=3691831371; auth=41yAG1RvddnKpurvZe6_JvusfPWFb0ci-HeJqQV2Rf8nOX0seYrMwizlRaa3j802MeX9_WxL6YM2NSh8YR3zTmgdMjyO23STt8e4rWIoRUxgiQs_nF_Hovy-tVjyGubYZKlNT9suLWjkUKfDSfppVQ; _fz_ssn=1658294788431610449'}

# create DataFrame for statement
if not os.path.isfile(mypath + "statement.csv"):    
    url = 'https://www.mql5.com/en/signals/'+ idlist[0] + '/export/positions'
    response = requests.get(url, headers=headers)

    state_df = pd.read_csv(io.BytesIO(response.content), delimiter=';')
    state_df.insert(loc=0, column='id', value=int(idlist[0]))

    state_df.to_csv(mypath + "statement.csv")
    time.sleep(random.randint(3, 10))



fileIds = [f.split('.')[0] for f in listdir(mypath) if isfile(join(mypath, f))]
statment_count = len(fileIds)
# state_count = len(statistics_df)

for id in idlist[:5]:
    # pass
    # load mt5 statements
    state_df = pd.read_csv(mypath + "statement.csv", index_col=[0])
    if int(id) not in  state_df.id.values:
        url = 'https://www.mql5.com/en/signals/' + id + '/export/positions'
        response = requests.get(url, headers=headers)  
        
        temp = pd.read_csv(io.BytesIO(response.content), delimiter=';')
        temp.insert(loc=0, column='id', value=int(id))

        state_df = pd.concat([state_df, temp])
        
        state_df.to_csv(mypath + "statement.csv")
    
        print ('Loaded: ', statment_count, end="\r")
        statment_count += 1    
        time.sleep(random.randint(3, 10))
        
    
    # load mt5 Statistics
#     statistics_df = pd.read_csv(mypath + "statistics.csv", index_col=[0])
#     if int(id) not in  statistics_df.id.values:
#         url = 'https://www.mql5.com/en/signals/' + id + '?source=Site+Signals+MT5+Tile'
#         response = requests.get(url, headers=headers)  
#         htmlContent = response.content
#         soup = bs.BeautifulSoup(htmlContent,'html.parser')

#         labels = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__label")]
#         values = [x.get_text() for x in  soup.find_all("div", class_="s-data-columns__value")]

#         dic = {}
#         dic['id'] = int(id)
#         for i in range(len(labels)):
#             dic[labels[i].strip().split(':')[0]] = values[i].strip()
#         temp_df = pd.DataFrame(dic, index=[0])
#         statistics_df = pd.concat([statistics_df, temp_df])
        
#         statistics_df.to_csv(mypath + "statistics.csv")
        
#         print ('Loaded: ', len(statistics_df), end="\r")
#         time.sleep(random.randint(3, 10))    

        


In [36]:
pd.read_csv('Data/statement.csv', index_col=[0])#.id.unique()

,id,Time,Type,Volume,Symbol,Price,Time.1,Price.1,Commission,Swap,Profit
0,1487129,2022.08.12 10:26:01,Sell,0.12,EURUSD,1.02988,2022.08.12 10:49:40,1.02922,-0.84,NaN,7.92
1,1487129,2022.08.11 10:21:05,Buy,0.12,EURUSD,1.03246,2022.08.11 11:58:19,1.03380,-0.84,NaN,16.08
2,1487129,2022.08.10 11:42:42,Buy,0.12,EURUSD,1.02237,2022.08.10 12:31:15,1.02302,-0.84,NaN,7.80
3,1487129,2022.08.09 11:00:52,Buy,0.12,EURUSD,1.02159,2022.08.09 11:06:29,1.02230,-0.84,NaN,8.52
4,1487129,2022.08.08 10:31:54,Buy,0.12,EURUSD,1.01973,2022.08.08 10:38:04,1.02021,-0.84,NaN,5.76
...,...,...,...,...,...,...,...,...,...,...,...
6152,798045,2020.01.07 08:00:05,Buy,0.03,EURUSD,1.11834,2020.01.07 08:45:02,1.11874,NaN,NaN,1.20
6153,798045,2019.12.31 10:48:32,Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.00
6154,798045,2019.12.10 14:18:35,Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10073.00
6155,798045,2019.12.10 08:46:45,Credit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.00
